In [1]:
import warnings
warnings.filterwarnings("ignore")

# 교차검증과 그리드 서치
- 머신러닝을 사용할때 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법.
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd
wine = pd.read_csv('../Data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# Feature , Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

6:2:2 = tr : val : test

# 검증 세트 추가

In [4]:
# 전체 세트 중 훈련 세트와 테스트 세트를 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2 , random_state=42)

In [5]:
# 훈련 세트 중 훈련 세트와 검증 세트를 8:2의 기준으로 분리한다. 
sub_input , val_input , sub_target , val_target = train_test_split(train_input , train_target , test_size= 0.2, random_state=42)

In [6]:
# 훈련세트, 검증세트, 테스트 세트의 크기 구하기
print("훈련세트 : ", sub_input.shape)
print("검증세트 :", val_input.shape)
print("테스트 세트 : ", test_input.shape)

훈련세트 :  (4157, 3)
검증세트 : (1040, 3)
테스트 세트 :  (1300, 3)


In [7]:
# 훈련 세트와 검증 세트를 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("Train score: ", dt.score(sub_input, sub_target))
print("Vaild score: ", dt.score(val_input, val_target))

Train score:  0.9971133028626413
Vaild score:  0.864423076923077


---
# 교차검증

In [9]:
# 라이브러리 사용 시에 학습용, 검증용 나누지 않아도 된다.
from sklearn.model_selection import cross_validate
scores = cross_validate(dt , train_input , train_target)
scores

{'fit_time': array([0.00717092, 0.0067358 , 0.00733376, 0.0058949 , 0.00716805]),
 'score_time': array([0.00067711, 0.00074315, 0.0007863 , 0.00063109, 0.0008688 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [11]:
import numpy as np
np.mean(scores['test_score'])
# 정확(예측?)도가 85퍼 정도 이다.

0.855300214703487

---
# KFold : 분할기를 사용한 교차 검증

In [20]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=5) # 기본값 n_splits = 5
scores = cross_validate(dt, train_input , train_target , cv = splitter)
scores

{'fit_time': array([0.00652003, 0.00645924, 0.006706  , 0.00687313, 0.00918031]),
 'score_time': array([0.00286508, 0.00127196, 0.00061417, 0.00077701, 0.0006218 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [21]:
np.mean(scores['test_score'])
# n_splits를 5로 했을때, cross_validate 실행값과 같다. cross_validate의 기본값이 5이다.

0.855300214703487

In [28]:
# KFold의 Fold를 10개로 나누어서 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle= True , random_state=42) # shuffle : 랜덤하게 섞어주는게 과대적합 방지에 좋다
scores = cross_validate(dt, train_input , train_target , cv = splitter)
scores
np.mean(scores['test_score'])

0.8574181117533719

In [29]:
splitter = StratifiedKFold(n_splits=10) # 
scores = cross_validate(dt, train_input , train_target , cv = splitter)
scores
np.mean(scores['test_score'])

0.8616407292129834

---